In [ ]:
%pip install langchain langchain_community

In [ ]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
%pip install faiss-cpu

#Import the necessary libraries

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

#Set the GOOGLE GEMINI api key
[link to download api key](https://aistudio.google.com/app/apikey)

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


#Set the LLM and embedding models

In [ ]:


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
embeddings=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

## Step 1: Load and Process Documents
the document is about RAG

In [ ]:
# Step 1: Load and Process Documents
loader = TextLoader("/content/rag_report.txt")  # Replace with your file path
documents = loader.load()

In [ ]:
dict(documents[0])

{'id': None,
 'metadata': {'source': '/content/rag_report.txt'},
 'page_content': '# Report on Retrieval-Augmented Generation (RAG) and Its Use Cases\n\n## Introduction\n\nRetrieval-Augmented Generation (RAG) is a state-of-the-art approach that combines the strengths of information retrieval and generative modeling. RAG enables the generation of highly accurate, contextually relevant, and up-to-date content by integrating a retrieval mechanism with a generative language model. This technique has gained significant traction in various industries due to its ability to leverage external knowledge sources to augment generative tasks.\n\n---\n\n## What is RAG?\n\n### Definition\n\nRAG is a hybrid framework that uses a retriever to fetch relevant information from a knowledge base or external data source and then incorporates this retrieved information into the generation process of a language model.\n\n### Components of RAG\n\n1. **Retriever**: The retriever identifies relevant documents or 


# Step 2: Split Text into Manageable Chunks

In [ ]:

# Step 2: Split Text into Manageable Chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)

#Step 3: Create the vectorestore

In [ ]:


  # Replace with your preferred embedding model
vectorstore = FAISS.from_documents(chunks, embeddings)

# Step 4: Create a Retriever

In [ ]:
# Step 4: Create a Retriever
retriever = vectorstore.as_retriever()

#Step 5 Create the RAG Pipeline

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Use the following context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

# Step 6: Query the RAG System

In [ ]:
# Step 6: Query the RAG System
query = "What is use case of RAG?"  # Replace with your question
result = qa_chain({"query": query})

# Step 7: Display the Results
print("Answer:", result["result"])
print("\nSource Documents:")
for doc in result["source_documents"]:
    print(f"- {doc.page_content}")


<ipython-input-31-0564493d8d77>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Answer: Based on the provided context, here are some use cases of RAG (Retrieval-Augmented Generation):

*   **Customer Support:** Providing accurate, context-sensitive responses to customer queries using FAQs, knowledge bases, and product manuals.
*   **Legal Document Assistance:** Assisting lawyers by retrieving case laws, statutes, and precedents for drafting documents and preparing for cases.
*   **Scientific Research:** (While not explicitly detailed, it's implied that RAG can be used in scientific research)
*   **E-Commerce:** Improving product discovery and customer engagement by retrieving product specifications, reviews, and related items.
*   **Healthcare:** Supporting medical professionals in diagnosing conditions by retrieving patient history and medical literature.
*   **Business Intelligence:** Generating insights by retrieving and analyzing reports, market trends, and financial data.


Source Documents:
- ### 6. Healthcare
Healthcare applications of RAG include supportin